In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize_scalar
from obj_function import obj_form, grad_obj_form
from parepy_toolbox.pare import deterministic_algorithm_structural_analysis

In [2]:
# from parepy_toolbox.distributions import non_normal_approach_normal

# params_normal = {'loc': 0, 'scale': 1}
# result_normal = non_normal_approach_normal(1.3065, 'normal', params_normal)
# print(result_normal)

# params_gumbel_max = {'loc': 0.887487, 'scale': 5.1302}
# result_gumbel_max = non_normal_approach_normal(1.3065, 'gumbel max', params_gumbel_max)
# print(result_gumbel_max)

# params_log_normal = {'loc': 0.887487, 'scale': 5.1302}
# result_log_normal = non_normal_approach_normal(1.3065, 'log normal', params_log_normal)
# print(result_log_normal)

In [3]:
# # Dataset
# f = {'type': 'normal', 'parameters': {'mean': 40, 'sigma': 5.}, 'stochastic variable': False}
# p = {'type': 'normal', 'parameters': {'mean': 50, 'sigma': 2.5}, 'stochastic variable': False}
# w = {'type': 'normal', 'parameters': {'mean': 1000, 'sigma': 200}, 'stochastic variable': False}
# var = [f, p, w]

# # PAREpy setup
# setup = {
#                 'tolerance': 1e-6, 
#                 'max iterations': 20,
#                 'numerical model': 'fosm', 
#                 'variables settings': var, 
#                 'number of state limit functions or constraints': 1, 
#                 'none variable': None,
#                 'objective function': obj,
#                 'gradient objective function': grad_obj,
#                 'name simulation': None,
#         }

# def deterministic_algorithm_structural_analysis(setup: dict) -> tuple[pd.DataFrame, float, int]:
#     try:
#         if not isinstance(setup, dict):
#             raise TypeError('The setup parameter must be a dictionary.')
        
#         required_keys = [
#             'tolerance', 'max iterations', 'numerical model', 'variables settings',
#             'number of state limit functions or constraints', 'none variable',
#             'objective function', 'gradient objective function', 'name simulation'
#         ]
        
#         for key in required_keys:
#             if key not in setup:
#                 raise ValueError(f'The setup parameter must have the key: {key}.')
        
#         variables = setup['variables settings']
#         if not isinstance(variables, list):
#             raise TypeError('The "variables settings" must be a list.')
        
#         for i, var in enumerate(variables):
#             if not isinstance(var, dict):
#                 raise TypeError('Each variable in "variables settings" must be a dictionary.')
            
#             if 'parameters' not in var or not isinstance(var['parameters'], dict):
#                 raise ValueError('Each variable must have a "parameters" key with a dictionary value.')
            
#             if 'mean' not in var['parameters'] or 'sigma' not in var['parameters']:
#                 raise ValueError('Each variable must have "mean" and "sigma" in its parameters.')
            
#             if 'type' not in var:
#                 raise ValueError('Each variable must have a "type" key.')
            
#             if var['type'] not in ['normal', 'lognormal', 'gumbel max', 'gumbel min']:
#                 raise ValueError('The variable type must be one of: "normal", "lognormal", "gumbel max", "gumbel min".')
        

#         mu = []
#         sigma = []
#         tol = setup['tolerance']
#         max_iter = setup['max iterations']
#         none_variable = setup['none variable']
#         obj = setup['objective function']
#         grad_obj = setup['gradient objective function']
#         params_adapt = {}
        
#         for i, var in enumerate(variables):
#             mean = var['parameters']['mean']
#             std = var['parameters']['sigma']
#             mu.append(mean)
#             sigma.append(std)
            
#             if var['type'] == 'normal':
#                 params_adapt[f'var{i}'] = {
#                     'type': 'normal',
#                     'params': {
#                         'mu': mean,
#                         'sigma': std
#                     }
#                 }
#             elif var['type'] == 'lognormal':
#                 epsilon = np.sqrt(np.log(1 + (std / mean) ** 2))
#                 lambdaa = np.log(mean) - 0.5 * epsilon ** 2
#                 params_adapt[f'var{i}'] = {
#                     'type': 'lognormal',
#                     'params': {
#                         'lambda': float(lambdaa),
#                         'epsilon': float(epsilon)
#                     }
#                 }
#             elif var['type'] == 'gumbel max':
#                 gamma = 0.577215665  
#                 beta = np.pi / (np.sqrt(6) * std)
#                 alpha = mean - gamma / beta
#                 params_adapt[f'var{i}'] = {
#                     'type': 'gumbel max',
#                     'params': {
#                         'alpha': float(alpha),
#                         'beta': float(beta)
#                     }
#                 }
#             elif var['type'] == 'gumbel min':
#                 gamma = 0.577215665 
#                 beta = np.pi / (np.sqrt(6) * std)
#                 alpha = mean + gamma / beta
#                 params_adapt[f'var{i}'] = {
#                     'type': 'gumbel min',
#                     'params': {
#                         'alpha': float(alpha),
#                         'beta': float(beta)
#                     }
#                 }
                 

#         # for index, value in params_adapt.items():
#         #     print(f"index: {index}, \nvalue: {value}")

#         # print(f"mu: {mean}, \nsigma: {std}, \ntol: {tol}, \nmax_iter: {max_iter}, \nnone_variable: {none_variable}")
        
#         # Fixed in this algorithm
#         beta_list = [10000]
#         error = 1000
#         iter = 0
#         step = 1

#         x = np.transpose(np.array([mu.copy()]))
#         mu = x.copy()
#         jacobian_xy = np.diag(sigma)
#         jacobian_xy_trans = np.transpose(jacobian_xy)
#         jacobian_yx = np.linalg.inv(jacobian_xy)
#         y = jacobian_yx @ (x - mu)
#         x = jacobian_xy @ y + mu

#         while (error > tol and iter < max_iter):
#             beta = np.linalg.norm(y)
#             beta_list.append(beta)
#             g_y = obj(x.flatten().tolist())
#             grad_g_x = grad_obj(x.flatten().tolist())
#             grad_g_y = np.dot(jacobian_xy_trans, np.transpose(np.array([grad_g_x])))
#             num = (np.transpose(grad_g_y) @ y - g_y)
#             norm = np.linalg.norm(grad_g_y)
#             norm2 = norm ** 2
#             #alpha = grad_g_y / norm
#             #aux = g_y / norm
#             #y = -alpha * (beta + aux)
#             d = grad_g_y @ (num / norm2) - y
#             #step = minimize_scalar(f_alpha, bounds=(.001, 1), args=([y, d]), method='bounded')
#             #print(step.x)
#             #y += step.x * d
#             y += step * d
#             error = np.abs(beta_list[iter + 1] - beta_list[iter])
#             x = jacobian_xy @ y + mu

#             aux = {
#                 'iteration': iter,
#                 **{f'x_{i}': float(x_value.item()) for i, x_value in enumerate(x)},
#                 'error': error,
#                 'beta': beta
#             }
#             if iter == 0:
#                 results_df = pd.DataFrame([aux])
#             else:
#                 results_df = pd.concat([results_df, pd.DataFrame([aux])], ignore_index=True)

#             iter += 1
#         pf = pf_equation(beta)
        
#         return results_df, float(pf), float(beta)
            
#     except (Exception, TypeError, ValueError) as e:
#         print(f"Error: {e}")
#         return None, None, None

In [4]:
# Dataset
f = {'type': 'lognormal', 'parameters': {'mean': 40, 'sigma': 5.}, 'stochastic variable': False}
p = {'type': 'gumbel max', 'parameters': {'mean': 50, 'sigma': 2.5}, 'stochastic variable': False}
w = {'type': 'normal', 'parameters': {'mean': 1000, 'sigma': 200}, 'stochastic variable': False}
var = [f, p, w]

# PAREpy setup
setup = {
                'tolerance': 1e-6, 
                'max iterations': 20,
                'numerical model': 'form', 
                'variables settings': var, 
                'number of state limit functions or constraints': 1, 
                'none variable': None,
                'objective function': obj_form,
                'gradient objective function': grad_obj_form,
                'name simulation': None,
        }

# Call algorithm
df, pf, beta = deterministic_algorithm_structural_analysis(setup)
print(f"Probability of failure: {pf}, beta: {beta}")
df

Probability of failure: 0.0011477416542393648, beta: 3.0490734771714223


,iteration,x_0,x_1,x_2,error,beta
0,0,28.888889,47.777778,1355.555556,1.000000e+04,0.000000
1,1,28.610608,48.278348,1381.412191,2.981424e+00,2.981424
2,2,28.554623,48.304316,1379.312978,6.814406e-02,3.049568
3,3,28.550849,48.307987,1379.234050,4.895082e-04,3.049079
4,4,28.550395,48.308272,1379.220262,4.993079e-06,3.049074
5,5,28.550357,48.308303,1379.219296,4.732587e-08,3.049073
